In [2]:
# IMPORTS
import pandas as pd
import math
import os.path
import time
# from bitmex import bitmex
from binance.client import Client
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm_notebook #(Optional, used for progress-bars)

### API
# bitmex_api_key = '[REDACTED]'    #Enter your own API-key here
# bitmex_api_secret = '[REDACTED]' #Enter your own API-secret here
binance_api_key = '[REDACTED]'    #Enter your own API-key here
binance_api_secret = '[REDACTED]' #Enter your own API-secret here

### CONSTANTS
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440}
batch_size = 750
# bitmex_client = bitmex(test=False, api_key=bitmex_api_key, api_secret=bitmex_api_secret)
binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)


### FUNCTIONS
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": old = datetime.strptime('1 Jan 2017', '%d %b %Y')
#    elif source == "bitmex": old = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=False).result()[0][0]['timestamp']
    if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
#    if source == "bitmex": new = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=True).result()[0][0]['timestamp']
    return old, new

def get_all_binance(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "binance")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    if oldest_point == datetime.strptime('1 Jan 2017', '%d %b %Y'): print('Downloading all available %s data for %s. Be patient..!' % (kline_size, symbol))
    else: print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data.' % (delta_min, symbol, available_data, kline_size))
    klines = binance_client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = data_df.append(temp_df)
    else: data_df = data
    data_df.set_index('timestamp', inplace=True)
    if save: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

'''
def get_all_bitmex(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "bitmex")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    rounds = math.ceil(available_data / batch_size)
    if rounds > 0:
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data in %d rounds.' % (delta_min, symbol, available_data, kline_size, rounds))
        for round_num in tqdm_notebook(range(rounds)):
            time.sleep(1)
            new_time = (oldest_point + timedelta(minutes = round_num * batch_size * binsizes[kline_size]))
            data = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=batch_size, startTime = new_time).result()[0]
            temp_df = pd.DataFrame(data)
            data_df = data_df.append(temp_df)
    data_df.set_index('timestamp', inplace=True)
    if save and rounds > 0: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df
'''

'\ndef get_all_bitmex(symbol, kline_size, save = False):\n    filename = \'%s-%s-data.csv\' % (symbol, kline_size)\n    if os.path.isfile(filename): data_df = pd.read_csv(filename)\n    else: data_df = pd.DataFrame()\n    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "bitmex")\n    delta_min = (newest_point - oldest_point).total_seconds()/60\n    available_data = math.ceil(delta_min/binsizes[kline_size])\n    rounds = math.ceil(available_data / batch_size)\n    if rounds > 0:\n        print(\'Downloading %d minutes of new data available for %s, i.e. %d instances of %s data in %d rounds.\' % (delta_min, symbol, available_data, kline_size, rounds))\n        for round_num in tqdm_notebook(range(rounds)):\n            time.sleep(1)\n            new_time = (oldest_point + timedelta(minutes = round_num * batch_size * binsizes[kline_size]))\n            data = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=batch_size, st

In [3]:
data = get_all_binance('BTCUSDT', '5m', save = True)

All caught up..!


In [11]:
data.tail(10)

,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
timestamp,,,,,,,,,,,
2023-04-14 02:00:00,30775.34000000,30786.73000000,30732.27000000,30741.56000000,319.43946000,1681437899999,9823984.38442960,6384,131.30099000,4037588.66038750,0
2023-04-14 02:05:00,30741.57000000,30758.82000000,30719.27000000,30738.62000000,363.11627000,1681438199999,11161000.54147700,5905,170.55342000,5241929.01995940,0
2023-04-14 02:10:00,30738.62000000,30808.78000000,30738.62000000,30807.92000000,282.90970000,1681438499999,8706450.63440480,5088,143.56101000,4417914.41517760,0
2023-04-14 02:15:00,30807.93000000,30807.93000000,30756.18000000,30766.00000000,173.69602000,1681438799999,5346130.94534010,4080,72.20530000,2222206.10580950,0
2023-04-14 02:20:00,30766.00000000,30768.21000000,30733.36000000,30738.30000000,166.84198000,1681439099999,5129884.66339110,3921,72.33371000,2224068.94965110,0
2023-04-14 02:25:00,30738.30000000,30766.71000000,30733.33000000,30737.98000000,139.65550000,1681439399999,4293959.42130310,3544,67.79831000,2084393.01202610,0
2023-04-14 02:30:00,30737.99000000,30754.60000000,30735.00000000,30750.13000000,166.02573000,1681439699999,5104060.05883130,3327,100.94321000,3103277.16395160,0
2023-04-14 02:35:00,30750.13000000,30750.14000000,30692.27000000,30694.31000000,264.07029000,1681439999999,8113514.94802450,5165,56.39167000,1732740.66740330,0
2023-04-14 02:40:00,30694.31000000,30705.00000000,30635.00000000,30654.48000000,398.30212000,1681440299999,12215738.91651690,7233,134.21588000,4116077.85528610,0


In [24]:
def get_binance_data(symbol = 'BTCUSDT', timeframe = '1m', start_date = '01 Mar 2023 01:00:00', end_date = '02 Mar 2023 01:00:00'):
    klines = binance_client.get_historical_klines(symbol, timeframe, start_date, end_date)
    df = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    return df



In [26]:
crypto_data = get_binance_data('SOLUSDT', '15m', '01 Apr 2023', '14 Apr 2023 01:00:00')


In [27]:
crypto_data.head()

,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
timestamp,,,,,,,,,,,
2023-04-01 00:00:00,21.15000000,21.25000000,21.14000000,21.24000000,38950.23000000,1680308099999,825790.39460000,1473,18576.36000000,394026.25680000,0
2023-04-01 00:15:00,21.23000000,21.31000000,21.22000000,21.26000000,20724.72000000,1680308999999,440776.97100000,989,9589.81000000,203916.80850000,0
2023-04-01 00:30:00,21.25000000,21.28000000,21.10000000,21.17000000,30813.89000000,1680309899999,652878.65210000,1348,15790.57000000,334227.19220000,0
2023-04-01 00:45:00,21.17000000,21.19000000,21.13000000,21.16000000,11261.31000000,1680310799999,238196.64480000,535,4976.47000000,105263.38270000,0
2023-04-01 01:00:00,21.17000000,21.18000000,21.04000000,21.09000000,30043.54000000,1680311699999,633879.32100000,1652,11270.44000000,237859.68070000,0
